<a href="https://colab.research.google.com/github/demolakstate/bumble_bees_detection/blob/main/Siamese_Neural_Network_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install wandb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import random
import numpy as np
import keras
#import wandb
#from wandb.keras import WandbCallback
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Concatenate, Dot, Lambda, Input
from keras.datasets import mnist
#from keras.optimizers import Adam
import matplotlib.pyplot as plt

In [4]:
import numpy as np
import random
import os
import cv2
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array

In [5]:
from os import path, listdir

In [6]:
import tensorflow as tf


In [7]:
batch_size = 2608
img_height = 180
img_width = 180

In [8]:
dir = "/content/drive/MyDrive/BeeMachine/dataset_2/training2"

In [9]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 3260 files belonging to 2 classes.
Using 2608 files for training.


In [10]:
class_names = train_ds.class_names
print(class_names)

['Bombus_affinis', 'Bombus_auricomus']


In [11]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(2608, 180, 180, 3)
(2608,)


In [12]:
type(image_batch)

tensorflow.python.framework.ops.EagerTensor

In [13]:
#train_image_directory = "/content/drive/MyDrive/BeeMachine/dataset_2/training2"

In [14]:
def create_pairs(X,Y, num_classes):
    pairs, labels = [], []
    # index of images in X and Y for each class
    class_idx = [np.where(Y==i)[0] for i in range(num_classes)]
    # The minimum number of images across all classes
    min_images = min(len(class_idx[i]) for i in range(num_classes)) - 1
  
    for c in range(num_classes):
        for n in range(min_images):
            # create positive pair
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[c][n+1]]
            pairs.append((img1, img2))
            labels.append(1.0)
      
            # create negative pair
            neg_list = list(range(num_classes))
            neg_list.remove(c)
            # select a random class from the negative list. 
            # this class will be used to form the negative pair
            neg_c = random.sample(neg_list,1)[0]
            img1 = X[class_idx[c][n]]
            img2 = X[class_idx[neg_c][n]]
            pairs.append((img1,img2))
            labels.append(0.0)

    return np.array(pairs), np.array(labels)

In [15]:
pairs_train, labels_train = create_pairs(image_batch, labels_batch, 2)

In [16]:
pairs_train[2,0]
labels_train[0]

1.0

In [17]:
def euclidean_distance(vectors):
    vector1, vector2 = vectors
    sum_square = K.sum(K.square(vector1 - vector2), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def contrastive_loss(Y_true, D):
    margin = 1
    return K.mean(Y_true*K.square(D)+(1 - Y_true)*K.maximum((margin-D),0))

def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [18]:
def create_shared_network(input_shape):
    model = Sequential(name='Shared_Conv_Network')
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', 
                     input_shape=input_shape))
    model.add(MaxPooling2D())
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(units=128, activation='sigmoid'))
    return model

In [19]:
'''
Main code for training a Siamese neural network for face recognition
'''
# import utils
import numpy as np
from keras.layers import Input, Lambda
from keras.models import Model

#faces_dir = 'att_faces/'

# Import Training and Testing Data
#(X_train, Y_train), (X_test, Y_test) = get_data(dir)
X_train = image_batch
Y_train = labels_batch
num_classes = len(np.unique(Y_train))
print(f'num_classes: {num_classes}')

# Create Siamese Neural Network
input_shape = X_train.shape[1:]
#input_shape = X_train[0].shape
shared_network = create_shared_network(input_shape)
input_top = Input(shape=input_shape)
input_bottom = Input(shape=input_shape)
output_top = shared_network(input_top)
output_bottom = shared_network(input_bottom)
distance = Lambda(euclidean_distance, output_shape=(1,))([output_top, output_bottom])
model = Model(inputs=[input_top, input_bottom], outputs=distance)

# Train the model
training_pairs, training_labels = create_pairs(X_train, Y_train, 
                                num_classes=num_classes)
model.compile(loss=contrastive_loss, optimizer='adam',
              metrics=[accuracy])



num_classes: 2


In [20]:
input_shape

TensorShape([180, 180, 3])

In [21]:
X_train.shape[1:]

TensorShape([180, 180, 3])

In [22]:
#training_pairs_float = tf.cast(training_pairs, tf.float32)

In [23]:
type(training_labels[0])

numpy.float64

In [24]:
type(training_pairs[0][0][0][0][0])

numpy.float32

In [25]:
model.fit([training_pairs[:, 0], training_pairs[:, 1]], training_labels,
          batch_size=16,
          epochs=10)

Epoch 1/10
297/297 [==============================] - 20s 56ms/step - loss: 0.5262 - accuracy: 0.4989
Epoch 2/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.4999
Epoch 3/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.5001
Epoch 4/10
297/297 [==============================] - 16s 56ms/step - loss: 0.4998 - accuracy: 0.5001
Epoch 5/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.5000
Epoch 6/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.5001
Epoch 7/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.5000
Epoch 8/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.4999
Epoch 9/10
297/297 [==============================] - 16s 55ms/step - loss: 0.4998 - accuracy: 0.5000
Epoch 10/10
297/297 [==============================] - 16s 55ms/step - loss: 0.499

In [26]:

# Save the model
model.save('siamese_nn.h5')